# PyTorch：自动求导(Autograd)

在上面的例子里，需要我们手动实现神经网络的前向和后向传播。对于简单的两层网络，手动实现前向、后向传播不是什么难事，但是对于大型的复杂网络就比较麻烦了。 

庆幸的是，我们可以使用[自动微分](https://en.wikipedia.org/wiki/Automatic_differentiation)来自动完成神经网络中反向传播的计算。PyTorch中**autograd**包提供的正是这个功能。当使用autograd时，网络前向传播将定义一个**计算图**；图中的节点是tensor，边是函数，这些函数是输出tensor到输入tensor的映射。这张计算图使得在网络中反向传播时梯度的计算十分简单。 

这听起来复杂，但是实际操作很简单。如果我们想计算某些的tensor的梯度，我们只需要在建立这个tensor时加入这么一句：`requires_grad=True`。这个tensor上的任何PyTorch的操作都将构造一个计算图，从而允许我们稍后在图中执行反向传播。如果这个tensor`x`的`requires_grad=True`，那么反向传播之后`x.grad`将会是另一个张量，其为`x`关于某个标量值的梯度。

有时可能希望防止PyTorch在`requires_grad=True`的张量执行某些操作时构建计算图；例如，在训练神经网络时，我们通常不希望通过权重更新步骤进行反向传播。在这种情况下，我们可以使用`torch.no_grad()`上下文管理器来防止构造计算图。

下面我们使用PyTorch的Tensors和autograd来实现我们的两层的神经网络；我们不再需要手动执行网络的反向传播：

In [1]:
# 可运行代码见本文件夹中的 two_layer_net_autograd.py
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

# N是批大小；D_in是输入维度；
# H是隐藏层维度；D_out是输出维度  
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出数据
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重tensor，将requires_grad设置为True意味着我们希望在反向传播时候计算这些值的梯度
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):

    # 前向传播：使用tensor的操作计算预测值y。
    # 由于w1和w2有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 计算并输出loss，loss是一个形状为()的张量，loss.item()是这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # 使用autograd计算反向传播。这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()


    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 反向传播之后手动置零梯度
        w1.grad.zero_()
        w2.grad.zero_()

0 36571340.0
1 42503180.0
2 53865712.0
3 57161024.0
4 42253456.0
5 19925168.0
6 7085042.0
7 2814085.0
8 1605472.625
9 1175494.25
10 949169.5
11 792956.25
12 672060.875
13 574474.0625
14 494362.5625
15 427843.8125
16 372209.25
17 325347.3125
18 285632.1875
19 251718.140625
20 222585.5
21 197616.8125
22 176061.296875
23 157286.59375
24 140862.28125
25 126421.234375
26 113701.5625
27 102474.765625
28 92534.0625
29 83699.015625
30 75827.375
31 68802.734375
32 62517.87890625
33 56886.125
34 51836.04296875
35 47293.59375
36 43197.8671875
37 39500.03125
38 36156.20703125
39 33127.109375
40 30384.4296875
41 27892.892578125
42 25625.916015625
43 23562.59375
44 21682.3828125
45 19966.796875
46 18400.140625
47 16969.197265625
48 15660.3369140625
49 14461.2177734375
50 13361.912109375
51 12354.501953125
52 11429.6669921875
53 10579.6640625
54 9798.1640625
55 9079.640625
56 8417.751953125
57 7807.7099609375
58 7245.70556640625
59 6727.41796875
60 6249.02392578125
61 5807.2451171875
62 5399.34912109

419 0.00019700356642715633
420 0.00019232419435866177
421 0.00018832969362847507
422 0.00018405595619697124
423 0.0001800678001018241
424 0.0001760434388415888
425 0.00017172269872389734
426 0.0001683460286585614
427 0.00016448635142296553
428 0.00016129731375258416
429 0.00015835577505640686
430 0.0001550164306536317
431 0.00015107729996088892
432 0.00014832080341875553
433 0.0001448635885026306
434 0.00014243439363781363
435 0.0001399570028297603
436 0.00013665464939549565
437 0.00013415161811280996
438 0.00013126878184266388
439 0.00012902816524729133
440 0.0001261321740457788
441 0.0001237181422766298
442 0.00012141861952841282
443 0.0001189551257994026
444 0.0001172839110950008
445 0.00011483496928121895
446 0.00011260794417466968
447 0.0001107747302739881
448 0.00010866869706660509
449 0.00010723875311668962
450 0.00010494494199519977
451 0.00010318480053683743
452 0.00010145656415261328
453 9.982420306187123e-05
454 9.78979078354314e-05
455 9.636655158828944e-05
456 9.4436793006